In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen

from tqdm import tqdm_notebook
import unidecode

In [2]:
df_debut = pd.read_csv("../data/debut_album_1118-utf8.csv")
df_debut.head()

,artist,album,genre,single_count,release_date
0,2Cellos,2Cellos,"Cello rock, classical crossover",3,"June 10, 2011"
1,Deep Dark Robot,8 Songs About a Girl,"Post-grunge, punk blues",0,"MarchÂ 22,Â 2011Â (2011-03-22)"
2,The Young Professionals,9am to 5pm 5pm to Whenever\r(09:00 to 17:00 17...,NaN,0,"September 12, 2011 (Israel)\rJune 18, 2012 (in..."
3,Flash Bang Grenada,10 Haters,Alternative hip hop,0,"AugustÂ 23,Â 2011Â (2011-08-23)"
4,Her Majesty & the Wolves,111,"Dance, pop",3,"July 11, 2011 (UK)"


In [4]:
def get_xxl_articles():
    
    df_articles = pd.DataFrame(columns = ['artist', 'album', 'xxl_frequency'])
    
    for i in tqdm_notebook(range(len(list(df_debut['artist'])))):
        
        artist = df_debut['artist'][i]
        album = df_debut['album'][i]

        artist_ = artist.replace(" ", "%20")
        album_ = album.replace(" ", "%20")

        keyword = artist_ + "%20" + album_

        url = 'http://www.xxlmag.com/rest/carbon/api/searchfor/widgetsearch/?s=' + keyword + '&searchpage=1'
        response = requests.get(url).json()

        articles = response['data']['carbonwidget/searchnative-1']['params']['total']

        data = {
                'artist': artist,
                'album' : album,
                'xxl_frequency' : articles
            }

#         print(data)

        df_articles.loc[len(df_articles)] = data
    
    return df_articles

In [5]:
xxl_articles = get_xxl_articles()

HBox(children=(IntProgress(value=0, max=1951), HTML(value='')))

In [6]:
print(xxl_articles.shape)
xxl_articles.head()

(1951, 3)


,artist,album,xxl_frequency
0,2Cellos,2Cellos,0
1,Deep Dark Robot,8 Songs About a Girl,14
2,The Young Professionals,9am to 5pm 5pm to Whenever\r(09:00 to 17:00 17...,0
3,Flash Bang Grenada,10 Haters,0
4,Her Majesty & the Wolves,111,5423


In [7]:
def decodeuni(str):
    return unidecode.unidecode(str)

In [9]:
xxl_articles['artist'] = xxl_articles['artist'].apply(lambda x: unidecode.unidecode(x))
xxl_articles['album'] = xxl_articles['album'].apply(lambda x: unidecode.unidecode(x))
xxl_articles['artist'] = xxl_articles['artist'].apply(lambda x : x.replace(",", ""))
xxl_articles['album'] = xxl_articles['album'].apply(lambda x : x.replace(",", ""))

In [12]:
xxl_articles.loc[2, 'album'] = '9am to 5pm 5pm to Whenever'
xxl_articles.loc[529, 'album'] = 'If You Love Me'
xxl_articles.loc[1143, 'album'] = 'Gover Tela Body Language'
xxl_articles.loc[1411, 'album'] = 'Fresh Filter Volumn 1'

In [13]:
xxl_articles.to_csv("../data/buzz_xxl.csv", index=False, encoding='UTF-8')

In [14]:
df = pd.read_csv("../data/buzz_xxl-utf8.csv", encoding="UTF-8")

In [15]:
print(df.shape)
df.head()

(1951, 3)


,artist,album,xxl_frequency
0,2Cellos,2Cellos,0
1,Deep Dark Robot,8 Songs About a Girl,14
2,The Young Professionals,9am to 5pm 5pm to Whenever,0
3,Flash Bang Grenada,10 Haters,0
4,Her Majesty & the Wolves,111,5423


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951 entries, 0 to 1950
Data columns (total 3 columns):
artist           1951 non-null object
album            1951 non-null object
xxl_frequency    1951 non-null int64
dtypes: int64(1), object(2)
memory usage: 45.8+ KB


### Upload to MySQL database

In [17]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [18]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [19]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [20]:
df.to_sql(name="buzz_xxl", con=engine, if_exists='replace')